In [1]:
import json
from pygrok import Grok
from tqdm import tqdm
from collections import Counter
import operator
import pandas as pd
import string
import preprocessor as p
from nltk import bigrams, trigrams
from collections import defaultdict
import multiprocessing as mp,os

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


# NLTK Packages for Tokenization

In [2]:
import re, nltk, gensim, spacy
from textblob import TextBlob
import pickle as p

#### nltk tagger and punk download
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('brown')
nltk.download('wordnet')
nltk.download('stopwords')

### nltk packages
from nltk.data import load
from nltk.tokenize.casual import TweetTokenizer, casual_tokenize
from nltk.tokenize.mwe import MWETokenizer
from nltk.tokenize.punkt import PunktSentenceTokenizer
from nltk.tokenize.regexp import (
    RegexpTokenizer,
    WhitespaceTokenizer,
    BlanklineTokenizer,
    WordPunctTokenizer,
    wordpunct_tokenize,
    regexp_tokenize,
    blankline_tokenize,
)
from nltk.tokenize.repp import ReppTokenizer
from nltk.tokenize.sexpr import SExprTokenizer, sexpr_tokenize
from nltk.tokenize.simple import (
    SpaceTokenizer,
    TabTokenizer,
    LineTokenizer,
    line_tokenize,
)
from nltk.tokenize.texttiling import TextTilingTokenizer
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.tokenize.treebank import TreebankWordTokenizer
from nltk.tokenize.util import string_span_tokenize, regexp_span_tokenize
from nltk.tokenize.stanford_segmenter import StanfordSegmenter
from nltk.stem.porter import *

from nltk.corpus import stopwords 
import num2words

[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/ubuntu/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package brown to /home/ubuntu/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package wordnet to /home/ubuntu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Using Grok for string matching and clustering OS Logs to pre-defined rules

In [3]:
patterns = []
patterns.append('%{SYSLOGTIMESTAMP}%{SPACE}%{SYSLOGHOST}%{SPACE}avahi-autoipd.*Callout.*,%{SPACE}address%{SPACE}%{IP:src_ip_address}.*')# 0
patterns.append('%{SYSLOGTIMESTAMP}%{SPACE}%{SYSLOGHOST}%{SPACE}avahi-autoipd.*Starting.*%{SPACE}address%{SPACE}%{IP:src_ip_address}.*')# 1
patterns.append('%{SYSLOGTIMESTAMP}%{SPACE}%{SYSLOGHOST}%{SPACE}avahi-autoipd.*Successfully.*%{SPACE}address%{SPACE}%{IP:src_ip_address}.*')# 2
patterns.append('%{SYSLOGTIMESTAMP}%{SPACE}%{SYSLOGHOST}%{SPACE}.*Accepted%{SPACE}password%{SPACE}for%{SPACE}%{USER:local_user}%{SPACE}from%{SPACE}%{IP:src_ip_address}.*')# 3
patterns.append('%{SYSLOGTIMESTAMP}%{SPACE}%{SYSLOGHOST}%{SPACE}.*Accepted%{SPACE}password%{SPACE}for%{SPACE}%{SPACE}from%{SPACE}%{IP:src_ip_address}.*')# 4
patterns.append('%{SYSLOGTIMESTAMP}%{SPACE}%{SYSLOGHOST}%{SPACE}.*Accepted%{SPACE}publickey%{SPACE}for%{SPACE}%{USER:local_user}%{SPACE}from%{SPACE}%{IP:src_ip_address}.*')# 5
patterns.append('%{SYSLOGTIMESTAMP}%{SPACE}%{SYSLOGHOST}%{SPACE}.*Accepted%{SPACE}publickey%{SPACE}for%{SPACE}%{SPACE}from%{SPACE}%{IP:src_ip_address}.*')# 6
patterns.append('%{SYSLOGTIMESTAMP}%{SPACE}%{SYSLOGHOST}%{SPACE}.*Connection%{SPACE}closed%{SPACE}by%{SPACE}%{IP:src_ip_address}.*')# 7
patterns.append('%{SYSLOGTIMESTAMP}%{SPACE}%{SYSLOGHOST}%{SPACE}.*Connection%{SPACE}reset%{SPACE}by%{SPACE}%{IP:src_ip_address}.*')# 8
patterns.append('%{SYSLOGTIMESTAMP}%{SPACE}%{SYSLOGHOST}%{SPACE}.*Disconnected%{SPACE}from%{SPACE}%{IPORHOST:src_ip_address}.*')# 9
patterns.append('%{SYSLOGTIMESTAMP}%{SPACE}%{SYSLOGHOST}%{SPACE}.*error:%{SPACE}maximum%{SPACE}authentication%{SPACE}attempts%{SPACE}exceeded%{SPACE}for%{SPACE}%{USER:local_user}%{SPACE}from%{SPACE}%{IP:src_ip_address}.*')# 10
patterns.append('%{SYSLOGTIMESTAMP}%{SPACE}%{SYSLOGHOST}%{SPACE}.*error:%{SPACE}maximum%{SPACE}authentication%{SPACE}attempts%{SPACE}exceeded%{SPACE}for%{SPACE}%{SPACE}from%{SPACE}%{IP:src_ip_address}.*')# 11
patterns.append('%{SYSLOGTIMESTAMP}%{SPACE}%{SYSLOGHOST}%{SPACE}.*Failed%{SPACE}none%{SPACE}for%{SPACE}invalid%{SPACE}user%{SPACE}%{USER:local_user}%{SPACE}from%{SPACE}%{IP:src_ip_address}.*')# 12
patterns.append('%{SYSLOGTIMESTAMP}%{SPACE}%{SYSLOGHOST}%{SPACE}.*Failed%{SPACE}none%{SPACE}for%{SPACE}invalid%{SPACE}user%{SPACE}%{SPACE}from%{SPACE}%{IP:src_ip_address}.*')# 13
patterns.append('%{SYSLOGTIMESTAMP}%{SPACE}%{SYSLOGHOST}%{SPACE}.*Failed%{SPACE}password%{SPACE}for%{SPACE}%{USER:local_user}%{SPACE}from%{SPACE}%{IP:src_ip_address}.*')# 14
patterns.append('%{SYSLOGTIMESTAMP}%{SPACE}%{SYSLOGHOST}%{SPACE}.*Failed%{SPACE}password%{SPACE}for%{SPACE}%{SPACE}from%{SPACE}%{IP:src_ip_address}.*')# 15
patterns.append('%{SYSLOGTIMESTAMP}%{SPACE}%{SYSLOGHOST}%{SPACE}.*fatal:%{SPACE}packet_write_wait:%{SPACE}Connection%{SPACE}from%{SPACE}%{IP:src_ip_address}.*')# 16
patterns.append('%{SYSLOGTIMESTAMP}%{SPACE}%{SYSLOGHOST}%{SPACE}.*:%{SPACE}input_userauth_request: invalid%{SPACE}user%{SPACE}%{IP:src_ip_address}.*')# 17 
patterns.append('%{SYSLOGTIMESTAMP}%{SPACE}%{SYSLOGHOST}%{SPACE}.*:%{SPACE}PAM.*authentication%{SPACE}failure.*%{IP:src_ip_address}%{SPACE}%{USER:local_user}.*')# 18
patterns.append('%{SYSLOGTIMESTAMP}%{SPACE}%{SYSLOGHOST}%{SPACE}.*:%{SPACE}PAM.*authentication%{SPACE}failure.*%{IP:src_ip_address}%{SPACE}.*')# 19
patterns.append('%{SYSLOGTIMESTAMP}%{SPACE}%{SYSLOGHOST}%{SPACE}.*:%{SPACE}pam_unix\(sshd:auth\).*authentication%{SPACE}failure.*%{IP:src_ip_address}%{SPACE}%{USER:local_user}.*')# 20
patterns.append('%{SYSLOGTIMESTAMP}%{SPACE}%{SYSLOGHOST}%{SPACE}.*:%{SPACE}pam_unix\(sshd:auth\).*authentication%{SPACE}failure.*%{IP:src_ip_address}%{SPACE}.*')# 21
patterns.append('%{SYSLOGTIMESTAMP}%{SPACE}%{SYSLOGHOST}%{SPACE}.*:%{SPACE}Received%{SPACE}disconnect%{SPACE}from%{SPACE}%{IP:src_ip_address}.*')# 22
patterns.append('%{SYSLOGTIMESTAMP}%{SPACE}%{SYSLOGHOST}%{SPACE}.*fatal:%{SPACE}ssh_dispatch_run_fatal:%{SPACE}Connection%{SPACE}from%{SPACE}%{IP:src_ip_address}.*')# 23
patterns.append('%{SYSLOGTIMESTAMP}%{SPACE}%{SYSLOGHOST}%{SPACE}.*fatal:%{SPACE}Unable%{SPACE}to%{SPACE}negotiate%{SPACE}with%{SPACE}%{IP:src_ip_address}.*') # 24

In [4]:
def process(line, annotated_ips_l1):
    annotation = dict()
    for j, ptrn in enumerate(patterns):
        gork_match = Grok(ptrn).match(line)
        if (gork_match != None):
            try: 
                annotation['local_user'] = gork_match['local_user'] 
            except:
                pass
            
            annotation['src_ip_address'] =  gork_match['src_ip_address'] 
            annotation['action'] = j
            annotated_ips_l1.append(annotation)
            break
    
    return annotated_ips_l1


def process_wrapper(chunkStart, chunkSize,abs_file_path):
    with open(abs_file_path) as f:
        annotated_ips_l2=[]
        annotated_ips_l1=[]
        result_l2 = []
        f.seek(chunkStart)
        lines = f.read(chunkSize).splitlines()
        for line in lines:
            result_l2 = process(line,annotated_ips_l1)
        #print(result_l2)
        annotated_ips_l2.extend(result_l2)
        return annotated_ips_l2


def chunkify(fname,size=4096):
    fileEnd = os.path.getsize(fname)
    with open(fname,'rb') as f:
        chunkEnd = f.tell()
        while True:
            chunkStart = chunkEnd
            f.seek(size,1)
            f.readline()
            #print(var_1)
            #print(var_2)
            chunkEnd = f.tell()
            #print("Chunk Start: %s Chunk End: %s Size: %s" %(chunkStart,chunkEnd,chunkEnd-chunkStart))
            yield chunkStart, chunkEnd - chunkStart
            if chunkEnd > fileEnd:
                break

                
cores=90

pool = mp.Pool(cores)
jobs = []

abs_file_path = 'json_file_message.txt'
#create jobs
for chunkStart,chunkSize in chunkify(abs_file_path):
    #temp_words = []
    #print("Chunk-Start: %s Chunk-Size: %s" %(chunkStart,chunkSize))
    jobs.append( pool.apply_async(process_wrapper,(chunkStart,chunkSize,abs_file_path)) )


print("============== RESULTS =============")
results=[]
#wait for all jobs to finish
for job in tqdm(jobs):
    job.wait()
    #print(len(job.get()))
    job_result = job.get()
    results.extend(job_result)
    #print(len(results))


#clean up
pool.close()
#pool.join()

  0%|          | 0/334968 [00:00<?, ?it/s]

============== RESULTS =============


100%|██████████| 334968/334968 [3:55:29<00:00, 23.71it/s]


In [5]:
len(results)

10508887

In [15]:
result

{'action': 5, 'local_user': 'student', 'src_ip_address': '10.1.1.54'}

In [9]:
outF = open("logs_annotated_ip_usr.txt", "w")

for i, result in enumerate(results):
    line = str(result)
    outF.write(line)
    outF.write("\n")

outF.close()

In [10]:
with open("logs_annotated.json", "w") as output_file:
    json.dump(results, output_file)

In [12]:
import pickle

pickle.dump(results, open('result.pickle','wb'))

In [13]:
my_results_df = pd.DataFrame(results)
my_results_df.to_csv('logs_annotated.csv')

In [21]:
my_index = my_results_df.groupby('src_ip_address').count().index

In [26]:
cnt=0
for ip in my_index:
    if ip[:3] == '10.':
        cnt+=1

In [30]:
cnt

56

In [29]:
my_index[:56]

Index(['10.1.1.10', '10.1.1.11', '10.1.1.12', '10.1.1.13', '10.1.1.14',
       '10.1.1.15', '10.1.1.16', '10.1.1.17', '10.1.1.18', '10.1.1.19',
       '10.1.1.20', '10.1.1.21', '10.1.1.22', '10.1.1.23', '10.1.1.24',
       '10.1.1.25', '10.1.1.26', '10.1.1.28', '10.1.1.29', '10.1.1.30',
       '10.1.1.32', '10.1.1.33', '10.1.1.34', '10.1.1.35', '10.1.1.36',
       '10.1.1.37', '10.1.1.38', '10.1.1.39', '10.1.1.4', '10.1.1.40',
       '10.1.1.41', '10.1.1.42', '10.1.1.43', '10.1.1.44', '10.1.1.45',
       '10.1.1.46', '10.1.1.47', '10.1.1.48', '10.1.1.49', '10.1.1.5',
       '10.1.1.50', '10.1.1.51', '10.1.1.52', '10.1.1.53', '10.1.1.54',
       '10.1.1.55', '10.1.1.56', '10.1.1.57', '10.1.1.58', '10.1.1.6',
       '10.1.1.61', '10.1.1.64', '10.1.1.65', '10.1.1.7', '10.1.1.8',
       '10.1.1.9'],
      dtype='object', name='src_ip_address')